### 목적
- LangChain 동적으로 시스템 프롬프트 변경 -> dynamic_prompt 미들웨어 사용

In [3]:
import os
from dotenv import load_dotenv

from langchain.agents import create_agent
from langchain.tools import tool
from langchain_openai import ChatOpenAI

from typing import TypedDict

from langchain.agents.middleware import dynamic_prompt, ModelRequest
# dynamic_prompt: 실행 시점에 프롬프트를 동적으로 수정하는 미들웨어 데코레이터 -> 모델 호출 전 
# ModelRequest: LLM에게 보낼 요청(프롬프트, 파라미터 등)을 담는 객체 -> LLM 호출 직전

load_dotenv()

True

In [4]:
llm = ChatOpenAI(
    model='gpt-4o-mini',
    api_key=os.getenv("OPENAI_API_KEY")
)

class Context(TypedDict):
    user_role: str

@dynamic_prompt
def user_role_prompt(request: ModelRequest) -> str:
    """
    사용자 역할에 따라 시스템 프롬프트를 생성하세요.
    """
    user_role = request.runtime.context.get('user_role', 'user')
    base_prompt = '너는 유용한 어시스턴트이다.'

    if user_role == 'expert':
        return f'{base_prompt} 자세한 기술적 답변을 제공하세요.'
    elif user_role == 'beginner':
        return f'{base_prompt} 개념을 간단하게 설명하고 전문 용어는 피하세요.'

    return base_prompt

agent = create_agent(
    model=llm,
    tools=[],
    middleware=[user_role_prompt],
    context_schema=Context
) 
# context_schema를 명시하여 내부 구조를 보장시킨다. 
# -> 넣지 않으면 None이나 키 에러 위험이 있음

result = agent.invoke(
    {'messages': [{'role': 'user', 'content': '머신러닝에 대해서 설명해라.'}]},
    context=Context(user_role='expert')
)

print(result.get('messages')[-1].content)

머신러닝(Machine Learning)은 인공지능(AI)의 한 분야로, 컴퓨터가 명시적으로 프로그래밍되지 않고도 데이터를 통해 학습하고 예측할 수 있도록 하는 알고리즘과 기법들을 포함합니다. 머신러닝은 일반적으로 다음과 같은 세 가지 주요 범주로 나눌 수 있습니다.

1. **지도 학습(Supervised Learning)**:
   - 입력 데이터와 그에 대한 정답(label)이 주어질 때, 모델이 패턴을 학습하여 새로운 데이터에 대한 예측을 수행하는 방법입니다.
   - 예: 스팸 이메일 분류, 주택 가격 예측 등.
   - 주로 분류(Classification)와 회귀(Regression) 문제로 나뉩니다.

2. **비지도 학습(Unsupervised Learning)**:
   - 입력 데이터만 주어지고, 정답이 없는 경우에 데이터를 분석하여 숨겨진 구조나 패턴을 찾는 방법입니다.
   - 예: 고객 세분화, 차원 축소 등.
   - 클러스터링(Clustering)과 차원 축소(Dimensionality Reduction)의 기법이 포함됩니다.

3. **강화 학습(Reinforcement Learning)**:
   - 에이전트가 환경과 상호작용하며 보상을 최대화하기 위한 전략을 학습하는 방법입니다. 에이전트는 시도를 통해 얻은 보상 정보를 기반으로 행동을 조정합니다.
   - 예: 게임 플레이, 로봇 제어 등.

### 머신러닝 알고리즘
머신러닝에는 여러 가지 알고리즘이 있으며, 주요 알고리즘은 다음과 같습니다:

- **선형 회귀(Linear Regression)**: 연속적인 타겟 변수를 예측하는 데 사용.
- **로지스틱 회귀(Logistic Regression)**: 이진 분류 문제를 해결하는 데 사용.
- **결정 트리(Decision Tree)**: 데이터의 특징에 따라 분기를 생성해 예측.
- **서포트 벡터 머신(Support Vector Machine, SVM)**: 데이터를 분리하는 최적의 경계를 찾는 군집화 알고리즘.
- **

#### 미들웨어 총 정리
- dynamic_prompt: context, 사용자 상태, 시간 등에 따라 system prompt 수정
- wrap_model_call: LLM에게 보낼 요청 또는 응답을 가로채서 수정/검증/단축 종료
- wrap_tool_call: 에이전트가 선택한 툴 호출의 입력/출력을 감싸서 처리

#### 실행 순서
1. 사용자 입력
2. @dynamic_prompt -> 시스템 프롬프트 생성
3. @wrap_model_call -> LLM 호출 전후 개입 (prompt, 모델 교체 등)
4. LLM 판단 (어떤 툴 쓸지 결정)
5. @wrap_tool_call -> 선택된 툴 실행 전후 개입
6. 툴 결과 Observation 생성
7. @wrap_model_call -> 다시 LLM 호출 (결과 요약 또는 최종 응답 생성)
8. 최종 출력